<a href="https://colab.research.google.com/github/ZarakiKanzaki/ProjectLunarML/blob/main/ExploratoryDataAnalysisScryfall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exploratory Data Analysis

### What's Scryfall?
Scryfall is a search engine for Magic cards. The website Scryfall.com was introduced on the World Wide Web in October 2016, claiming to be faster than similar sites. It also advertises as mobile-friendly, comprehensive, timely, and powerful. Subsequent development focused on expanding the database to include high-resolution scans and previously uncatalogued game features such as artwork, card backs, tokens, and extras.

In [11]:
!pip install requests
import requests
import json

url = "https://api.scryfall.com/bulk-data/oracle_cards"

response = requests.get(url)




In [13]:
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data.get("download_uri"), indent=2))



"https://data.scryfall.io/oracle-cards/oracle-cards-20231009090140.json"


In [20]:
    download_uri = data.get("download_uri")
    if download_uri:
        download_response = requests.get(download_uri)

        if download_response.status_code == 200:
            with open("bulk-files.json", "wb") as file:
                file.write(download_response.content)
            print("File downloaded successfully.")
        else:
            print(f"Failed to download file. Status code: {download_response.status_code}")
    else:
        print("No 'download_uri' found in the JSON data.")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

File downloaded successfully.


The first thing we need to do with these bulk files is to clean them with pandas

In [46]:
import pandas as pd

df = pd.read_json("bulk-files.json")

df = df[df['border_color'] != 'silver ']
desired_categories = ['normal', 'saga', 'meld', "prototype", "transform", "split", "adventure", "flip", "modal_dfc", "leveler", "class"]
df = df[df['layout'].isin(desired_categories)]

df = df.drop(["oracle_id","multiverse_ids","mtgo_id","mtgo_foil_id","tcgplayer_id","cardmarket_id","released_at","uri","scryfall_uri","highres_image","image_status","image_uris","lang","cmc","colors","color_identity","legalities","reserved","foil","nonfoil","finishes","oversized","promo","games",
"reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri","collector_number","digital","rarity","card_back_id","artist","artist_ids","illustration_id","flavor_text",
"frame","full_art","textless","booster","story_spotlight","edhrec_rank","prices","related_uris","purchase_uris","security_stamp","preview","penny_rank","arena_id","all_parts","frame_effects","watermark","produced_mana","card_faces","tcgplayer_etched_id","promo_types",
"life_modifier","hand_modifier","attraction_lights","color_indicator","content_warning"], axis=1)



In [50]:
df.head()

,object,id,name,layout,mana_cost,type_line,oracle_text,keywords,border_color,power,toughness,loyalty
0,card,86bf43b1-8d4e-4759-bb2d-0b2e03ba7012,Static Orb,normal,{3},Artifact,"As long as Static Orb is untapped, players can...",[],white,NaN,NaN,NaN
1,card,7050735c-b232-47a6-a342-01795bfd0d46,Sensory Deprivation,normal,{U},Enchantment — Aura,Enchant creature\nEnchanted creature gets -3/-0.,[Enchant],black,NaN,NaN,NaN
2,card,e718b21b-46d1-4844-985c-52745657b1ac,Road of Return,normal,{G}{G},Sorcery,Choose one —\n• Return target permanent card f...,[Entwine],black,NaN,NaN,NaN
3,card,036ef8c9-72ac-46ce-af07-83b79d736538,Storm Crow,normal,{1}{U},Creature — Bird,Flying (This creature can't be blocked except ...,[Flying],white,1,2,NaN
4,card,b125d1e7-5d9b-4997-88b0-71bdfc19c6f2,Walking Sponge,normal,{1}{U},Creature — Sponge,{T}: Target creature loses your choice of flyi...,[],black,1,1,NaN


In [48]:
df.describe()

,object,id,name,layout,mana_cost,type_line,oracle_text,keywords,border_color,power,toughness,loyalty
count,27434,27434,27434,27434,27049,27434,26809,27434,27434,14709,14709,291
unique,1,27434,27385,11,968,3061,25179,1971,4,29,32,11
top,card,86bf43b1-8d4e-4759-bb2d-0b2e03ba7012,Ineffable Blessing,normal,,Instant,,[],black,2,2,4
freq,27434,1,6,26630,1246,3026,335,14886,26260,4416,3666,104


In [51]:
prototyped = df[df['layout'] == 'prototype']
prototyped.head()

,object,id,name,layout,mana_cost,type_line,oracle_text,keywords,border_color,power,toughness,loyalty
288,card,9d927dfd-ae66-4235-a61b-39c85d0c1222,Goring Warplow,prototype,{6},Artifact Creature — Construct,Prototype {1}{B} — 1/1 (You may cast this spel...,"[Prototype, Deathtouch]",black,5,4,NaN
1436,card,74dfabfd-e13b-4512-a733-abe514be6404,Combat Thresher,prototype,{7},Artifact Creature — Construct,Prototype {2}{W} — 1/1 (You may cast this spel...,"[Double strike, Prototype]",black,3,3,NaN
2143,card,a0418156-bcba-4edc-8ea5-ce9ca9b979bd,Woodcaller Automaton,prototype,{10},Artifact Creature — Construct,Prototype {2}{G}{G} — 3/3 (You may cast this s...,[Prototype],black,8,8,NaN
3066,card,e4e5529a-848b-486c-97fb-1def35df7837,Blitz Automaton,prototype,{7},Artifact Creature — Construct,Prototype {2}{R} — 3/2 (You may cast this spel...,"[Haste, Prototype]",black,6,4,NaN
4001,card,50cc9b90-792e-4cf7-ab8c-cb616d94092a,Cradle Clearcutter,prototype,{6},Artifact Creature — Golem,Prototype {2}{G} — 1/3 (You may cast this spel...,[Prototype],black,3,6,NaN


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_25, *['index'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_26, *['index'], **{})
chart

import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(_df_27, *['id'], **{})
chart

import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(_df_28, *['name'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def violin_plot(df, value_colname, facet_colname, figscale=1, mpl_palette_name='Dark2', **kwargs):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (12 * figscale, 1.2 * figscale * len(df[facet_colname].unique()))
  plt.figure(figsize=figsize)
  sns.violinplot(df, x=value_colname, y=facet_colname, palette=mpl_palette_name, **kwargs)
  sns.despine(top=True, right=True, bottom=True, left=True)
  return autoviz.MplChart.from_current_mpl_state()

chart = violin_plot(_df_38, *['index', 'type_line'], **{'inner': 'stick'})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_39, *['index', 'count()', 'id'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_40, *['index', 'count()', 'name'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_41, *['index', 'count()', 'mana_cost'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_42, *['index', 'count()', 'type_line'], **{})
chart

In [49]:
print("Number of elements in the DataFrame:", df.size)

Number of elements in the DataFrame: 329208
